#### Config ####

In [ ]:
#https://github.com/99designs/aws-vault
choco install aws-vault
start cmd -A "/k aws-vault add terraform_user"

#$env:AWS_ACCESS_KEY_ID = ""
#$env:AWS_SECRET_ACCESS_KEY = ""

In [ ]:
# Add folder with executables to PATH

$folderPath = "$(Split-Path -Path (Get-Location) -Parent)\_tools"

$oldPath = [System.Environment]::GetEnvironmentVariable('Path', 'User')

if (-not $oldPath.Contains($folderPath)) {
    $newPath = "$oldPath        ;$folderPath"
    [System.Environment]::SetEnvironmentVariable('Path', $newPath, 'User')
    Write-Host "New PATH: $newPath"

} else {
    Write-Host "The folder is already in the PATH."
}

#### Initialize ####

In [ ]:
$ErrorActionPreference = "Inquire"

function tf {aws-vault exec terraform_user -- terraform $args}

function prompt_to_confirm { 
    $user_response = Read-host "Continue? y/n or 1/0"
    if ($user_response -ne "y" -and $user_response -ne 1) {break}
}

In [ ]:
$configurations = "$(Get-Location)\configurations"
cd $configurations

In [ ]:
$config = "hello-world-app"

In [ ]:
$config = "one_webserver"

In [ ]:
cd "$configurations\$config"

In [ ]:
(pwd).Path

In [ ]:
$aws_region = "us-east-2"
$env:AWS_REGION=$aws_region #for aws_vault
$env:TF_VAR_aws_region=$aws_region #for terraform

In [ ]:
terraform init #-upgrade

#### Workflow ####

In [ ]:
terraform fmt

In [ ]:
terraform validate

In [ ]:
tf plan -out=tfplan
terraform show -json tfplan > plan.json

In [ ]:
#Output planned resource addresses and their names in the cloud

$plan = Get-Content -Raw -Path "plan.json" | ConvertFrom-Json

$resourcesToDeploy = $plan.resource_changes | Where-Object { 
    $_.change.actions -contains "create" -or $_.change.actions -contains "update" 
} | ForEach-Object { 
    "Type: " + $_.type
    "TF name: " + $_.name
    "Cloud name: " + $_.change.after.name
    "Name tag: " + $_.change.after.tags_all.name
    ""
}

$resourcesToDeploy
Get-Date

In [ ]:
###      ###
### PLAN ###
###      ###

tf plan
Get-Date

In [ ]:
###       ###
### APPLY ###
###       ###

try {prompt_to_confirm} catch {break}
tf apply -auto-approve
Get-Date

In [ ]:
###         ###
### DESTROY ###
###         ###

try {prompt_to_confirm} catch {break}
tf destroy -auto-approve
Get-Date

#### Info ####

In [ ]:
terraform version

In [ ]:
#Root module configuration
cat .\main.tf

In [ ]:
($resources = terraform state list) | Out-Host

In [ ]:
foreach ($resource in $resources) { terraform state show $resource }

In [ ]:
terraform output

#### Remote access ####

In [ ]:
Test-NetConnection -ComputerName $public_ip -Port 22

In [53]:
puttygen "$keyname.pem" -O private -o "$keyname.ppk"

In [54]:
#Start SSH session

#Start ssh -A "ec2-user@$public_ip -i $keyname.pem"
Start "plink.exe" -A "-ssh -i $keyname.ppk ec2-user@$public_ip"

In [ ]:
#Execute single SSH command

#ssh ec2-user@$public_ip -i "$keyname.pem" 'cat /etc/xattr.conf'
plink -ssh -i "$keyname.ppk" "ec2-user@$public_ip" "cat /etc/xattr.conf"

#### Troubleshooting ####

In [ ]:
#$env:TF_LOG = "TRACE"
#$env:TF_LOG_PATH = "log.txt"
#$env:TF_LOG = ""

In [ ]:
tf apply -refresh-only -auto-approve